# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [126]:
import sqlite3
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [4]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [5]:
players_180_190 = int(player_data[(player_data.height >= 180) & (player_data.height <= 190)]['player_api_id'].count())

In [6]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [7]:
player_data['birthday'] = pd.to_datetime(player_data['birthday'])
players_1980 = int(player_data[player_data.birthday.dt.year == 1980]['player_api_id'].count())

In [8]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [9]:
highest_players = [i for i in player_data.sort_values(['weight', 'player_name'], ascending=[False, True]).iloc[:10]['player_name']]

In [10]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [11]:
cnt_player_year = player_data[(player_data.birthday.dt.year >= 1980) & (player_data.birthday.dt.year <= 1990)]\
                             .groupby(player_data.birthday.dt.year)['player_api_id'].count().reset_index()

years_born_players = [(int(row['birthday']), int(row['player_api_id'])) for idx, row in cnt_player_year.iterrows()]

In [12]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [13]:
adriano_mean = player_data[player_data.player_name.str.split(' ').str[0] == "Adriano"]['height'].mean()
adriano_std = player_data[player_data.player_name.str.split(' ').str[0] == "Adriano"]['height'].std()

In [14]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [15]:
week = ['Monday', 
        'Tuesday', 
        'Wednesday', 
        'Thursday',  
        'Friday', 
        'Saturday',
        'Sunday']

dow_with_min_players_born = week[player_data.groupby(player_data.birthday.dt.weekday)['player_api_id'].count().reset_index()\
                                            .sort_values(['player_api_id'])\
                                            .iloc[0]['birthday']]

In [16]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [17]:
league_data = pd.read_sql("SELECT * FROM League;", db)
match_data = pd.read_sql('''SELECT m.*, l.name
                            FROM Match m
                            JOIN League l ON l.country_id = m.country_id;''', db)

In [18]:
league_most_matches = str(match_data.groupby(match_data.name)['match_api_id'].count().reset_index()\
                                    .sort_values(['match_api_id', 'name'], ascending=[False, True])\
                                    .iloc[0]['name'])

In [19]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [20]:
columns_match = ['home_player_' + str(i) for i in range(1,12)] + ['away_player_' + str(i) for i in range(1,12)] + ['match_api_id']
player_match = match_data[columns_match].copy()

all_player_match = pd.DataFrame(columns=['player_api_id', 'match_api_id'])
for column in columns_match[:-1]:
    all_player_match = pd.concat([all_player_match, player_match[[column, 'match_api_id']].rename(columns={column: 'player_api_id'})], ignore_index=True)

all_player_match.dropna(inplace=True)

id_player = all_player_match.groupby(all_player_match.player_api_id)['match_api_id'].count().reset_index()\
                                                     .sort_values(['match_api_id'], ascending=False)\
                                                     .iloc[0]['player_api_id']

In [21]:
max_matches_player = player_data[player_data.player_api_id == int(id_player)]['player_name'].iloc[0]

In [22]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [275]:
player_attr_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)

In [276]:
column_attr = [i for i in player_attr_data.columns \
               if i not in ['id', 'player_fifa_api_id', 'player_api_id', \
                            'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']]

corr_player_attr = player_attr_data[column_attr].corr().reset_index()
corr_player_attr2 = pd.DataFrame(columns=['attrs', 'corr']) 
for idx, column in enumerate(column_attr[:-1]):
    df = corr_player_attr[['index', column]].iloc[idx+1:].rename(columns={column: 'corr'})
    df['attrs'] = df['index'].apply(lambda x: (x, column))
    df.drop('index', inplace=True, axis=1)
    corr_player_attr2 = pd.concat([corr_player_attr2, df], ignore_index=True)

In [277]:
top_correlated_features = [row['attrs'] for idx, row in corr_player_attr2.sort_values('corr', ascending=False).iloc[:5].iterrows()]

In [79]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [118]:
#player_data[player_data.player_name == 'Neymar'] 19533

In [147]:
def neymar_distance(player):
    square = np.square(neymar[column_attr].values - player[column_attr].values)
    sum_square = np.sum(square)
    distance = np.sqrt(sum_square)
    
    return distance


player_attr_data['date'] = pd.to_datetime(player_attr_data['date'])
player_late_date = pd.DataFrame(columns=['player_api_id'] + column_attr)

for player in [i for i in player_attr_data.player_api_id.unique()]:
    df = player_attr_data[player_attr_data.player_api_id == player]\
                       .sort_values('date', ascending=False).iloc[[0]][['player_api_id'] + column_attr]
    player_late_date = pd.concat([player_late_date, df])

    
neymar = player_late_date[player_late_date.player_api_id == 19533]
player_late_date.drop(player_late_date[player_late_date.player_api_id == 19533].index, inplace=True, axis=0)

player_late_date['neymar_distance'] = player_late_date.apply(neymar_distance, axis=1)

neymar_similarities = [row['player_name'] for idx, row in \
                       pd.merge(player_late_date, player_data[['player_api_id', 'player_name']], on='player_api_id') \
                       .dropna() \
                       .sort_values(['neymar_distance', 'player_name']).iloc[:5].iterrows()]

In [154]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [171]:
team_data = pd.read_sql("SELECT * FROM Team;", db)
match_home = pd.merge(match_data, team_data[['team_long_name', 'team_api_id']].rename(columns={'team_long_name':'team_long_name_home'}), \
                      left_on='home_team_api_id', right_on='team_api_id')



borussia_bundesliga_2008_2009_matches = int(match_home[(match_home.season == '2008/2009') \
                                                   & (match_home.name == 'Germany 1. Bundesliga') \
                                                   & (match_home.team_long_name_home == 'Borussia Dortmund')].count().iloc[0])

In [169]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [194]:
match_home_away = pd.merge(match_home, team_data[['team_long_name', 'team_api_id']].rename(columns={'team_long_name':'team_long_name_away'}), \
                      left_on='away_team_api_id', right_on='team_api_id')

match_home_away_bundesliga_2008_2009 = match_home_away[(match_home_away.season == '2008/2009') \
                                                        & (match_home_away.name == 'Germany 1. Bundesliga')]

cnt_home_bundesliga_2008_2009 = match_home_away_bundesliga_2008_2009.groupby('team_long_name_home')['match_api_id'].count().reset_index()
cnt_away_bundesliga_2008_2009 = match_home_away_bundesliga_2008_2009.groupby('team_long_name_away')['match_api_id'].count().reset_index()

In [195]:
total_away_home_bundesliga_2008_2009 = pd.merge(cnt_home_bundesliga_2008_2009.rename(columns={'match_api_id':'cnt_home'}), \
                                                cnt_away_bundesliga_2008_2009.rename(columns={'match_api_id':'cnt_away'}), \
                                                left_on='team_long_name_home', 
                                                right_on='team_long_name_away')
total_away_home_bundesliga_2008_2009['total'] = total_away_home_bundesliga_2008_2009['cnt_home'] + \
                                                total_away_home_bundesliga_2008_2009['cnt_away']

team_most_matches_bundesliga_2008_2009 = int(total_away_home_bundesliga_2008_2009['total'].max())

In [196]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [205]:
match_arsenal_2015_2016 = match_home_away[(match_home_away.season == '2015/2016') \
                          & (((match_home_away.team_long_name_home == 'Arsenal') & (match_home_away.home_team_goal > match_home_away.away_team_goal)) \
                          |  ((match_home_away.team_long_name_away == 'Arsenal') & (match_home_away.home_team_goal < match_home_away.away_team_goal)))]

arsenal_won_matches_2015_2016 = int(match_arsenal_2015_2016.count().iloc[0])

In [204]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [240]:
match_2015_2016 = match_home_away[(match_home_away.season == '2015/2016')]

In [241]:
cnt_home_2015_2016 = match_2015_2016.groupby('team_long_name_home')['match_api_id'].count().reset_index()
cnt_away_2015_2016 = match_2015_2016.groupby('team_long_name_away')['match_api_id'].count().reset_index()

total_away_home_2015_2016 = pd.merge(cnt_home_2015_2016.rename(columns={'match_api_id':'cnt_home'}), \
                                    cnt_away_2015_2016.rename(columns={'match_api_id':'cnt_away'}), \
                                    left_on='team_long_name_home', 
                                    right_on='team_long_name_away')

total_away_home_2015_2016['total'] = total_away_home_2015_2016['cnt_home'] + total_away_home_2015_2016['cnt_away']

In [242]:
match_2015_2016_win_home = match_2015_2016[(match_2015_2016.home_team_goal > match_2015_2016.away_team_goal)] \
                           .groupby('team_long_name_home')['match_api_id'].count().reset_index()

total_away_home_2015_2016 = pd.merge(total_away_home_2015_2016, \
                                     match_2015_2016_win_home.rename(columns={'match_api_id':'cnt_win_home'}), \
                                     on='team_long_name_home')

match_2015_2016_win_away = match_2015_2016[(match_2015_2016.home_team_goal < match_2015_2016.away_team_goal)] \
                           .groupby('team_long_name_away')['match_api_id'].count().reset_index()

total_away_home_2015_2016 = pd.merge(total_away_home_2015_2016, \
                                     match_2015_2016_win_away.rename(columns={'match_api_id':'cnt_win_away'}), \
                                     on='team_long_name_away')

total_away_home_2015_2016['total_win'] = total_away_home_2015_2016['cnt_win_away'] + total_away_home_2015_2016['cnt_win_home']
total_away_home_2015_2016['koef'] = total_away_home_2015_2016['total_win'] / total_away_home_2015_2016['total']

team_highest_winrate_2015_2016 = total_away_home_2015_2016 \
                                .sort_values(['koef', 'team_long_name_home'], ascending=[False, True])['team_long_name_home'] \
                                .iloc[0]

In [244]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [272]:
match_home_away['date'] = pd.to_datetime(match_home_away['date'])
match_data_england_2010_2011 = match_home_away[(match_home_away.season == '2010/2011') \
                                                & (match_home_away.name == 'England Premier League')]

In [273]:
highest_gap_england_2010_2011 = 0

for team in [i for i in match_data_england_2010_2011.team_long_name_home.unique()]:
    df = match_data_england_2010_2011[(match_data_england_2010_2011.team_long_name_home == team) \
                                      | (match_data_england_2010_2011.team_long_name_away == team)].sort_values('date')
          
    max_day = df['date'].diff().max().days
    if max_day > highest_gap_england_2010_2011:
        highest_gap_england_2010_2011 = max_day

In [274]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [37]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")